<a href="https://colab.research.google.com/github/gusper01/seminarioITBA/blob/gh-pages/seminarioITBA_PIPELINE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Configuración SPARK en COLAB**

In [1]:
#Se debe instalar JVM para usar spark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os # libreria de manejo del sistema operativo
os.system("wget -q https://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz")
os.system("tar xf /spark-2.4.5-bin-hadoop2.7.tgz")
# instalar pyspark
!pip install -q pyspark
# cargar data de prediccion (Generada en el colab de EDA)
# https://github.com/gusper01/seminarioITBA/blob/gh-pages/seminarioITBA_EDA.ipynb
os.system("wget -q https://raw.githubusercontent.com/gusper01/seminarioITBA/gh-pages/data/datatotal01102021v1.csv")

     |████████████████████████████████| 212.4 MB 55 kB/s 
     |████████████████████████████████| 198 kB 56.5 MB/s 


0

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("SeminarioITBA")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [3]:
spark

# **Creación Dataframe Spark con data de ML**

In [4]:
#pipeline_proc = "https://raw.githubusercontent.com/gusper01/seminarioITBA/gh-pages/data/datatotal01102021v.csv"
# Continuacion del Pipelie: Se genera Dataframe de Spark desde el dataset con la predicción producto del proceso
# de modelado de ML (se trae desde GITHUB )
p = "datatotal01102021v1.csv"
df_spark = spark.read.csv(p, header=True, inferSchema=True)


In [6]:
df_spark.printSchema()

root
 |-- MSSubClass: integer (nullable = true)
 |-- MSZoning: string (nullable = true)
 |-- LotFrontage: double (nullable = true)
 |-- LotArea: integer (nullable = true)
 |-- Street: string (nullable = true)
 |-- Alley: string (nullable = true)
 |-- LotShape: string (nullable = true)
 |-- LandContour: string (nullable = true)
 |-- Utilities: string (nullable = true)
 |-- LotConfig: string (nullable = true)
 |-- LandSlope: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Condition1: string (nullable = true)
 |-- Condition2: string (nullable = true)
 |-- BldgType: string (nullable = true)
 |-- HouseStyle: string (nullable = true)
 |-- OverallQual: integer (nullable = true)
 |-- OverallCond: integer (nullable = true)
 |-- YearBuilt: integer (nullable = true)
 |-- YearRemodAdd: integer (nullable = true)
 |-- RoofStyle: string (nullable = true)
 |-- RoofMatl: string (nullable = true)
 |-- Exterior1st: string (nullable = true)
 |-- Exterior2nd: string (nullable = tr

In [7]:
# Visualizar Dataframe Spark
df_spark.show()

+----------+--------+-----------+-------+------+-----+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+----------+-----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+------+-----+-----------+-------+------+------+--------+-------------+-----------+----+---------+
|MSSubClass|MSZoning|LotFrontage|LotArea|Street|Alley|LotShape|LandContour|Utilities|LotConfig|LandSlope|Neighborhood|Condition1|

# **Persistencia**
Generación de Archivo Parquet

In [5]:
# Se genera   Archivo Parquet
df_parquet = df_spark.write.parquet("/tmp/datatotal01102021.parquet")


In [6]:
# Consultas SQL sobre Archivo Parquet 
df_parquet_read = spark.read.parquet("/tmp/datatotal01102021.parquet")
df_parquet_read.createOrReplaceTempView("propiedades")
spark.sql("select count(Id) from propiedades").show()

+---------+
|count(Id)|
+---------+
|     1459|
+---------+



In [10]:
spark.sql("select * from propiedades where Neighborhood = 'Gilbert'").show(truncate=False)

+----------+--------+-----------+-------+------+-----+--------+-----------+---------+---------+---------+------------+----------+----------+--------+----------+-----------+-----------+---------+------------+---------+--------+-----------+-----------+----------+----------+---------+---------+----------+--------+--------+------------+------------+----------+------------+----------+---------+-----------+-------+---------+----------+----------+--------+--------+------------+---------+------------+------------+--------+--------+------------+------------+-----------+------------+----------+----------+-----------+----------+-----------+------------+----------+----------+----------+----------+----------+----------+-----------+-------------+---------+-----------+--------+------+-----+-----------+-------+------+------+--------+-------------+-----------+----+----------+
|MSSubClass|MSZoning|LotFrontage|LotArea|Street|Alley|LotShape|LandContour|Utilities|LotConfig|LandSlope|Neighborhood|Condition1

Se observa que en la tabla parquet los datos deberian reprocesarse por ejemplo la columna GarageYrBlt corresponde al año y esta en formato decimal "2005.0"